In [ ]:
!export HF_HUB_ENABLE_HF_TRANSFER=1
!pip -q install -U hf_transfer

import os, torch
from huggingface_hub import login, whoami

In [ ]:
BASE_MODEL  = "Qwen/Qwen3-Coder-30B-A3B-Instruct"
ADAPTER_ID  = "j05hr3d/LoRA-Q-30B-R8"

In [ ]:
# -------- User switches (edit these) ------------------------------------------
USE_DRIVE_CACHE   = False       # True = cache reused across sessions via Drive
DO_MERGE_ADAPTER  = True       # True = merge LoRA into base for faster inference
MAX_NEW_TOKENS    = 1024       # keep modest in Colab to avoid OOM
TEMPERATURE       = 0.7
TOP_P             = 0.8
TOP_K             = 20
REPETITION_PENALTY= 1.05

In [ ]:

%pip install -q transformers peft
%pip install -q transformers peft bitsandbytes accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Base model load
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# Load LoRA fine-tuned adapter on top of it
model = PeftModel.from_pretrained(base, ADAPTER_ID)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
model.eval()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 47.0 MB/s eta 0:00:00


config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00006-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3MoeForCausalLM(
      (model): Qwen3MoeModel(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-47): 48 x Qwen3MoeDecoderLayer(
            (self_attn): Qwen3MoeAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [ ]:
try:
    merged = model.merge_and_unload()
    model = merged
    print("[info] LoRA merged into base weights.")
except Exception as e:
    print(f"[note] merge_and_unload not performed: {e}")

[info] LoRA merged into base weights.


In [ ]:
import gc, torch
import textwrap # Import textwrap module
#gc.collect()
#torch.cuda.empty_cache()

# -------- Simple chat generation helper ---------------------------------------
def chat_generate(model, tokenizer, messages, max_new_tokens=1024, temperature=0.2, top_p=0.9, top_k=20, repitition_penalty=1.1):
    # prepare the model input
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature is not None and temperature > 0),
            temperature=TEMPERATURE,
            top_p=TOP_P,
            top_k=TOP_K,
            repetition_penalty=REPETITION_PENALTY,
            pad_token_id=tokenizer.pad_token_id,
        )
    gen_ids = out[0][inputs.input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True)

# -------- Example inference ---------------------------------------------------
#PROMPT      = "Write a quick sort algorithm in Python with brief comments."
PROMPT       = """### Instruction:
Write a Cryptol property that tests the function described below.
Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The code block should contain valid Cryptol source that can be loaded by the Cryptol tool and by SAW.

### Request:
Task: Using ONLY the functions and constraints provided, create a monomorphic Cryptol property named `equivalent` that proves decrypt inverts encrypt for a 512-byte plaintext.

```cryptol
encryptChar : [8] -> [8] -> [8]
encryptChar key c = (c + key)
decryptChar : [8] -> [8] -> [8]
decryptChar key c = (c - key)
encrypt : {n} [8] -> [n][8] -> [n][8]
encrypt key txt = [ encryptChar key c | c <- txt ]
decrypt : {n} [8] -> [n][8] -> [n][8]
decrypt key txt = [ decryptChar key c | c <- txt ]```"""

messages = [{"role": "user", "content": PROMPT}]
print("\n[generate] Running a sample …")
content = chat_generate(
    model, tokenizer, messages,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    top_k=TOP_K,
    repitition_penalty=REPETITION_PENALTY
)
print("\n=== OUTPUT ===\n")
print(textwrap.dedent(content))

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



[generate] Running a sample …

=== OUTPUT ===

```cryptol
equivalent : [512][8] -> [8] -> Bit
equivalent txt key = (decrypt key (encrypt key txt)) == txt
```


In [ ]:
import torch
import re
import textwrap # Import textwrap module
#gc.collect()
#torch.cuda.empty_cache()

# ------------------ Helpers -------------------------
def extract_code_block(text: str) -> str:
    """
    Get content inside ```cryptol ...``` or generic ```...```.
    If no fence is found, return the raw text.
    """
    m = re.search(r"```(?:cryptol)?\s*(.*?)```", text, flags=re.S | re.I)
    return (m.group(1) if m else text).strip()

FUNCTION_PROMPT_TEMPLATE: str = """### Instruction:
    Write a Cryptol function that implements the tasks described below
    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The file should be valid Cryptol source that can be loaded by the Cryptol tool and by SAW.

    ### Request:
    Task: {task}
    """

PROPERTY_PROMPT_TEMPLATE: str = """### Instruction:
    Write a Cryptol property that tests the function described below.
    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The code block should contain valid Cryptol source that can be loaded by the Cryptol tool and by SAW.

    ### Request:
    Task: {task}
    """

def generate_response(task, task_id, setup_code, type, enable_thinking=True):
    # -------- Prepare prompt --------
    if type == "property":
        prompt = PROPERTY_PROMPT_TEMPLATE.format(task=task)
    else:
        prompt = FUNCTION_PROMPT_TEMPLATE.format(task=task)
    if setup_code != "":
            prompt += f"\n### Additional setup code:\n```cryptol\n{setup_code}\n```"

    result_ = f"\n=== Task {task_id} ===\n"
    result_ += f"\n[PROMPT BEGIN]\n{prompt}[PROMPT END]\n\n"
    # -------- Inference --------
    content = chat_generate(
        model, tokenizer, [{"role": "user", "content": prompt}],
        max_new_tokens=1024,
        temperature=TEMPERATURE,
        top_p=TOP_P,
    )

    source_code = extract_code_block(
        content
        )
    if setup_code != "":
        source_code = f"{setup_code}\n\n{source_code}"
    result_ += f"\n[GENERATE BEGIN]\n```cryptol\n{source_code}\n```\n[GENERATE END]\n\n"
    print(result_)
    return result_

In [ ]:
response = chat_generate(
    model, tokenizer, [
        {"role": "user", "content": """### Instruction:
Write a SAW script that contains a specification for the addition of two 32 bit integers and proves the C function provided is equivalent to the generated specification.
Output exactly one code block in the format ```saw ...``` and nothing else — no explanation, no extra text. The code block must be valid SAW and directly runnable with `saw`.

### Constraints:
    - Include minimal helpful comments only.
    - Do not print or log anything.
    - The output must be a single ```saw ...``` block and nothing else.

### Context:
[C source: add.bc]
```c
#include <stdint.h>
uint32_t add(uint32_t x, uint32_t y) { return x + y; }
```
"""
    }],
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    repitition_penalty=1.05
)
'''
generate_response(
    task="Implement a function named `xor8` which takes two 8-bit words and returns their bitwise XOR. Also provide a constant `zero8` equal to 0 (8-bit).",
    task_id=1,
    setup_code="",
    type="function"
)
'''
print(response)

```saw
// Load the LLVM bytecode file
bc <- llvm_load_module "add.bc"

// Specify the addition function
spec_add <- llvm_spec bc "add" [llvm_int 32, llvm_int 32] (llvm_int 32)

// Prove equivalence between the specification and the C implementation
llvm_verify bc "add" [llvm_int 32, llvm_int 32] (llvm_int 32) spec_add
```


In [ ]:
generate_response(
    task="Implement a function named `xor8` which takes two 8-bit words and returns their bitwise XOR. Also provide a constant `zero8` equal to 0 (8-bit).",
    task_id=1,
    setup_code="",
    type="function"
)
generate_response(
    task="Implement a function named `matrix` which takes no input and returns a 3 x 3 matrix of tuples. For the function use a nested comprehension to write an expression to produce a 3 × 3 matrix (as a sequence of sequences), such that the ijth entry contains the value (i, j).",
    task_id=2,
    setup_code="",
    type="function"
)
generate_response(
    task="Implement a Caesar cipher. **Cryptol performs modulo arithmatic automatically**. At the minimum there must be a function `encrypt key text` that performs encryption on finite length strings and a function `decrypt key text` that performs decryption on finite length strings.",
    task_id=3,
    setup_code="",
    type="function"
)
generate_response(
    task="Using ONLY the functions and constraints provided, create a monomorphic Cryptol property named `equivalent` that proves decrypt inverts encrypt for every 8-bit key and 512-byte plaintext.",
    task_id=4,
    setup_code="encryptChar : [8] -> [8] -> [8]\nencryptChar key c = (c + key)\ndecryptChar : [8] -> [8] -> [8]\ndecryptChar key c = (c - key)\nencrypt : {n} [8] -> [n][8] -> [n][8]\nencrypt key txt = [ encryptChar key c | c <- txt ]\ndecrypt : {n} [8] -> [n][8] -> [n][8]\ndecrypt key txt = [ decryptChar key c | c <- txt ]",
    type="property"
)



=== Task 1 ===

[PROMPT BEGIN]
### Instruction:
    Write a Cryptol function that implements the tasks described below
    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The file should be valid Cryptol source that can be loaded by the Cryptol tool and by SAW.

    ### Request:
    Task: Implement a function named `xor8` which takes two 8-bit words and returns their bitwise XOR. Also provide a constant `zero8` equal to 0 (8-bit).
    [PROMPT END]


[GENERATE BEGIN]
```cryptol
xor8 : [8] -> [8] -> [8]
xor8 x y = x ^ y

zero8 : [8]
zero8 = 0
```
[GENERATE END]



=== Task 2 ===

[PROMPT BEGIN]
### Instruction:
    Write a Cryptol function that implements the tasks described below
    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The file should be valid Cryptol source that can be loaded by the Cryptol tool and by SAW.

    ### Request:
    Task: Implement a 

'\n=== Task 4 ===\n\n[PROMPT BEGIN]\n### Instruction:\n    Write a Cryptol property that tests the function described below.\n    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The code block should contain valid Cryptol source that can be loaded by the Cryptol tool and by SAW.\n\n    ### Request:\n    Task: Using ONLY the functions and constraints provided, create a monomorphic Cryptol property named `equivalent` that proves decrypt inverts encrypt for every 8-bit key and 512-byte plaintext.\n    \n### Additional setup code:\n```cryptol\nencryptChar : [8] -> [8] -> [8]\nencryptChar key c = (c + key)\ndecryptChar : [8] -> [8] -> [8]\ndecryptChar key c = (c - key)\nencrypt : {n} [8] -> [n][8] -> [n][8]\nencrypt key txt = [ encryptChar key c | c <- txt ]\ndecrypt : {n} [8] -> [n][8] -> [n][8]\ndecrypt key txt = [ decryptChar key c | c <- txt ]\n```[PROMPT END]\n\n\n[GENERATE BEGIN]\n```cryptol\nencryptChar : [8] -> [8] -> [

In [ ]:
generate_response(
    task="Implement a function named `xor8` which takes two 8-bit words and returns their bitwise XOR. Also provide a constant `zero8` equal to 0 (8-bit).",
    task_id=1,
    setup_code="",
    type="function"
)
generate_response(
    task="Implement a function named `matrix` which takes no input and returns a 3 x 3 matrix of tuples. For the function use a nested comprehension to write an expression to produce a 3 × 3 matrix (as a sequence of sequences), such that the ijth entry contains the value (i, j).",
    task_id=2,
    setup_code="",
    type="function"
)
generate_response(
    task="Implement a Caesar cipher. **Cryptol performs modulo arithmatic automatically**. At the minimum there must be a function `encrypt key text` that performs encryption on finite length strings and a function `decrypt key text` that performs decryption on finite length strings.",
    task_id=3,
    setup_code="",
    type="function"
)
generate_response(
    task="Using ONLY the functions and constraints provided, create a monomorphic Cryptol property named `equivalent` that proves decrypt inverts encrypt for every 8-bit key and 512-byte plaintext.",
    task_id=4,
    setup_code="encryptChar : [8] -> [8] -> [8]\nencryptChar key c = (c + key)\ndecryptChar : [8] -> [8] -> [8]\ndecryptChar key c = (c - key)\nencrypt : {n} [8] -> [n][8] -> [n][8]\nencrypt key txt = [ encryptChar key c | c <- txt ]\ndecrypt : {n} [8] -> [n][8] -> [n][8]\ndecrypt key txt = [ decryptChar key c | c <- txt ]",
    type="property"
)


=== Task 1 ===

[PROMPT BEGIN]
### Instruction:
    Write a Cryptol function that implements the tasks described below
    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The file should be valid Cryptol source that can be loaded by the Cryptol tool and by SAW.

    ### Request:
    Task: Implement a function named `xor8` which takes two 8-bit words and returns their bitwise XOR. Also provide a constant `zero8` equal to 0 (8-bit).
    [PROMPT END]


[GENERATE BEGIN]
```cryptol
xor8 : [8] -> [8] -> [8]
xor8 x y = x ^ y

zero8 : [8]
zero8 = 0
```
[GENERATE END]



=== Task 2 ===

[PROMPT BEGIN]
### Instruction:
    Write a Cryptol function that implements the tasks described below
    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The file should be valid Cryptol source that can be loaded by the Cryptol tool and by SAW.

    ### Request:
    Task: Implement a 

'\n=== Task 4 ===\n\n[PROMPT BEGIN]\n### Instruction:\n    Write a Cryptol property that tests the function described below.\n    Output exactly one code block in the format ```cryptol ...``` and nothing else — no explanation, no extra text. The code block should contain valid Cryptol source that can be loaded by the Cryptol tool and by SAW.\n\n    ### Request:\n    Task: Using ONLY the functions and constraints provided, create a monomorphic Cryptol property named `equivalent` that proves decrypt inverts encrypt for every 8-bit key and 512-byte plaintext.\n    \n### Additional setup code:\n```cryptol\nencryptChar : [8] -> [8] -> [8]\nencryptChar key c = (c + key)\ndecryptChar : [8] -> [8] -> [8]\ndecryptChar key c = (c - key)\nencrypt : {n} [8] -> [n][8] -> [n][8]\nencrypt key txt = [ encryptChar key c | c <- txt ]\ndecrypt : {n} [8] -> [n][8] -> [n][8]\ndecrypt key txt = [ decryptChar key c | c <- txt ]\n```[PROMPT END]\n\n\n[GENERATE BEGIN]\n```cryptol\nencryptChar : [8] -> [8] -> [